In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [31]:
fifa_dataset = pd.read_csv("data.csv") #reads the dataset
fifa_dataset.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [32]:
print(fifa_dataset.size)
print(fifa_dataset.shape)

1620423
(18207, 89)


In [33]:
fifa_dataset.info() #gives us the information of each column in the dataset and what kinds of vlaues it holds

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
Unnamed: 0                  18207 non-null int64
ID                          18207 non-null int64
Name                        18207 non-null object
Age                         18207 non-null int64
Photo                       18207 non-null object
Nationality                 18207 non-null object
Flag                        18207 non-null object
Overall                     18207 non-null int64
Potential                   18207 non-null int64
Club                        17966 non-null object
Club Logo                   18207 non-null object
Value                       18207 non-null object
Wage                        18207 non-null object
Special                     18207 non-null int64
Preferred Foot              18159 non-null object
International Reputation    18159 non-null float64
Weak Foot                   18159 non-null float64
Skill Moves                 18159 non-null fl

In [34]:
fifa_dataset.drop(columns=['Unnamed: 0','Photo','Nationality','Flag','Club Logo','Real Face','Joined','Loaned From','Contract Valid Until','Release Clause'],inplace=True)
print(fifa_dataset.size) #we drop the above fields because even without doing any calculaions, these attributes do not contribute to the value field that we want to predict
print(fifa_dataset.shape)

1438353
(18207, 79)


In [35]:
fifa_dataset.columns #after dropping we see the columns again

Index(['ID', 'Name', 'Age', 'Overall', 'Potential', 'Club', 'Value', 'Wage',
       'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Position', 'Jersey Number',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes'],
      dtype='object')

In [36]:
#Some of these attributes like the Value, wage, LS, LM, etc all have string values that can be converted into numerical values
#So we convert them to numerical values

def currency_converter(values): #example take the field of Value, Christiano Ronaldo is worth 77 million pouds. The worth is given with the currency symbol and an M next to the number. So we use that to convert the values
    if values[-1] == 'M':
        worth = values[1:-1]
        worth = float(worth) * 1000000
        return worth
    elif values[-1] == 'K':
        worth = values[1:-1]
        worth = float(worth) * 1000
        return worth
    else:
        return 0
    
fifa_dataset['Value'] = fifa_dataset['Value'].apply(currency_converter)
fifa_dataset['Wage'] = fifa_dataset['Wage'].apply(currency_converter)
#fifa_dataset.describe()

In [39]:
fifa_dataset.head()

,ID,Name,Age,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,158023,L. Messi,31,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,20801,Cristiano Ronaldo,33,94,94,Juventus,77000000.0,405000.0,2228,Right,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,190871,Neymar Jr,26,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,193080,De Gea,27,91,93,Manchester United,72000000.0,260000.0,1471,Right,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,192985,K. De Bruyne,27,91,92,Manchester City,102000000.0,355000.0,2281,Right,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [ ]:
#The fields LS, LM, ST etc are the differet positions in a team's formation. The values in the field mean that if you were to score the player's performance in that position, it would be that value
#Example, if you put Christiano Ronaldo as a LS, ST OR RS whihc stands for Left Striker, Striker and Right Striker respectively, then his score will be in the 90's because that is the position he plays
#But if you were to put him in LCB, CB or RCB which stands for Left Center Back, Center Back and Right Center Back, then he will have a very poor score of 47 because he is not trained to play in those postions
#The fields have values of two number with a plus sign, so we add those numbers and make them integers

In [38]:
#corr = st.spearmanr(fifa_dataset, fifa_dataset['Potential'])
#print(corr)